In [ ]:
import os
import healpy as hp

import numpy as np
import xarray as xr
from scipy import interpolate

In [ ]:
class HEALBIN:
    def __init__(self, nside):
        self.nside = nside
        nstep = int(np.log(nside) / np.log(2))
        lidx = np.arange(nside * nside, dtype="int")
        xx = np.zeros([nside * nside], dtype="int")
        yy = np.zeros([nside * nside], dtype="int")
        th, ph = hp.pix2ang(nside, np.arange(nside * nside), nest=True)
        # 1min
        # for i in range(nstep):
        #    xx = xx + (2**i) * (((lidx // (4**i)) % 4) % 2)
        #    yy = yy + (2**i) * (((lidx // (4**i)) % 4) // 2)
        i = [2**x for x in range(nstep)]
        ii = [x * x for x in i]
        for p in range(nstep):
            a, b = divmod(((lidx // (ii[p])) % 4), 2)
            xx += (i[p]) * b
            yy += (i[p]) * a
        self.lon = np.zeros([nside, nside])
        self.lat = np.zeros([nside, nside])
        self.lon[xx, yy] = -ph / np.pi * 180.0  # Q
        self.lat[xx, yy] = 90.0 - th / np.pi * 180.0
        self.lidx = lidx
        self.nstep = nstep
        self.xx = xx
        self.yy = yy
        self.th = th
        self.ph = ph

    def MARS2HEAL(self, marslon, marslat, rot=[0, 0]):
        theta = (90.0 - (marslat - rot[1])) / 180.0 * np.pi
        phi = -(marslon - rot[0]) / 180.0 * np.pi
        pix, weights = hp.get_interp_weights(self.nside, theta, phi, nest=True)
        pix -= (self.nside * self.nside) * (pix.min() // (self.nside * self.nside))

        return (
            self.xx[pix] * self.nside + self.yy[pix],
            weights,
            self.lon + rot[0],
            self.lat - rot[1],
        )

    def REBINHEAL(self, pix, weights, data, nside=-1):
        if nside == -1:
            nside = self.nside
        b = np.zeros([nside * nside])
        # b[:]=np.nan
        bh = np.zeros([nside * nside])
        # bh[:]=np.nan
        for iii in range(4):
            b = b + np.bincount(
                pix[iii, :],
                weights=weights[iii, :] * (data).flatten(),
                minlength=nside * nside,
            )
            bh = bh + np.bincount(
                pix[iii, :], weights=weights[iii, :], minlength=nside * nside
            )
        b[bh > 0] /= bh[bh > 0]
        b[bh == 0] = np.nan
        del bh
        res = b.reshape(nside, nside)
        del b
        return res

In [ ]:
%%time
tag_id = os.environ.get("tag_id", "SV_A11981")
model = os.environ.get("model", "merged")
healpy = os.environ.get("healpy", "4")

basepath = os.environ.get("basepath", "../data_local/")
basepath = os.environ.get("basepath", "/home/datawork-lops-iaocea/work/fish/marc/")

input_filename = basepath + "diff/" + tag_id + "-" + model + ".zarr"

nside = 4096 * 2 * 2
if healpy == "1":
    nside = 4096

output_filename = "../data_local/healpixbase_nside_" + str(nside) + "_" + model + ".nc"

tag_id, model, healpy, basepath, input_filename, nside, output_filename

In [ ]:
%%time
# Initialize the pixelisation (may take few seconds)

HB = HEALBIN(nside)
# grid size is 0.3970387705733788 Km
# CPU times: user 29.4 s, sys: 18.5 s, total: 47.9 s
# Wall time: 46.9 s


# 公称赤道半径 nominal equatorial Earth radius 6378.1 km（正確に）
# 公称極半径 nominal polar Earth radius 6356.8 km（正確に）
R = 6378.1
R = 6356.8
grid_size = R * np.sqrt(4 * np.pi / (12 * nside * nside))
grid_size

In [ ]:
ds = xr.open_dataset(
    input_filename, engine="zarr", chunks={"time": 1, "ni": -1, "nj": -1}
)
ds

In [ ]:
%%time
p, w, lon, lat = HB.MARS2HEAL(
    ds.longitude.data.compute()[:, :].flatten(),
    ds.latitude.data.compute()[:, :].flatten(),
    rot=[30, 0],
)

In [ ]:
# print((p//nside).min(),(p//nside).max(),(p%nside).min(),(p%nside).max())
nside2 = (
    max((p // nside).max() - (p // nside).min(), (p % nside).max() - (p % nside).min())
    + 10
)
nside2

In [ ]:
p2 = (p // nside - (p // nside).min()) * nside2 + (p % nside - (p % nside).min())
# plt.scatter((p2.T) % nside2, (p2.T) // nside2)

## prepare parameters 


In [ ]:
def regrid(pdf):
    return HB.REBINHEAL(p2, w, pdf, nside=nside2)

In [ ]:
H0 = xr.apply_ufunc(
    regrid,
    ds.H0.compute(),  # .isel(time=slice(0,2))
    input_core_dims=[["nj", "ni"]],
    output_core_dims=[["x", "y"]],
    exclude_dims=set(("nj", "ni")),
    vectorize=True,
    dask="parallelized",
    dask_gufunc_kwargs={
        "output_sizes": {"x": nside2, "y": nside2},
    },
    output_dtypes=[ds.H0.dtype],
)

In [ ]:
ds_healpy = H0.to_dataset(name="H0")
ds_healpy["longitude"] = xr.DataArray(
    data=lon[
        (p // nside).min() : (p // nside).min() + nside2,
        (p % nside).min() : (p % nside).min() + nside2,
    ],
    dims=["x", "y"],
)  # dict(lon
ds_healpy["latitude"] = xr.DataArray(
    data=lat[
        (p // nside).min() : (p // nside).min() + nside2,
        (p % nside).min() : (p % nside).min() + nside2,
    ],
    dims=["x", "y"],
)  # dict(lon
ds_healpy = (
    ds_healpy.assign_coords({"longitude": ds_healpy.longitude})
    .assign_coords({"latitude": ds_healpy.latitude})
    .assign_coords({"x": ds_healpy.x})
    .assign_coords({"y": ds_healpy.y})
    .assign_attrs({"grid_size": grid_size})
)
ds_healpy = ds_healpy.assign(
    {"p2": (["a", "b"], p2), "w": (["a", "b"], w), "nside2": nside2, "nside": nside}
)

In [ ]:
ds_healpy.to_netcdf(output_filename)